In [1]:
import cv2
import numpy as np
import os
import PIL
from PIL import Image
import random
from tqdm import tqdm
import pickle


import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
device=torch.device('cpu')

cpu


In [3]:
transformer=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize((0.5), (0.5))
])

In [4]:
train_path='C:/Users/joshu/OneDrive/A levels/Other/Crest/Crest code/imgRecognition/signal_frames/silhouettes'
test_path='C:/Users/joshu\OneDrive/A levels/Other/Crest/Crest code/imgRecognition/signal_frames/test_silhouettes'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [5]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['backwards', 'downwards', 'forwards', 'left', 'right', 'upwards']


In [6]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        #((width-kernel_size+2Padding)/stride) +1
        
        #Input shape= (64,1,150,150)
        #batch_size = 64
        #channels =1 hopefully not 3 fix?
        #height/width=150
        #stride=1
        #Padding=1
        #kernel_size(f)=3
        #(batch size,channels,height width)
        
        self.conv1=nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (64,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #normalisation
        #Shape= (64,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (64,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (64,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (64,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (64,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (64,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (64,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (64,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        #Feed forward function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        

In [7]:
model=ConvNet(num_classes=6).to(device)

In [8]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [9]:
num_epochs=10

In [10]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

4200 300


In [11]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
       
        #if torch.cuda.is_available():
        #    images=Variable(images.cuda())
        #    labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        #if torch.cuda.is_available():
        #    images=Variable(images.cuda())
        #    labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.pth')
        best_accuracy=test_accuracy
    
       

Epoch: 0 Train Loss: tensor(2.5859) Train Accuracy: 0.8861904761904762 Test Accuracy: 0.9866666666666667
Epoch: 1 Train Loss: tensor(0.0231) Train Accuracy: 0.9964285714285714 Test Accuracy: 0.99
Epoch: 2 Train Loss: tensor(0.0024) Train Accuracy: 0.9995238095238095 Test Accuracy: 0.9866666666666667
Epoch: 3 Train Loss: tensor(0.0267) Train Accuracy: 0.995 Test Accuracy: 0.9933333333333333
Epoch: 4 Train Loss: tensor(0.0240) Train Accuracy: 0.9973809523809524 Test Accuracy: 0.99
Epoch: 5 Train Loss: tensor(0.0328) Train Accuracy: 0.9957142857142857 Test Accuracy: 0.9466666666666667
Epoch: 6 Train Loss: tensor(0.0458) Train Accuracy: 0.9952380952380953 Test Accuracy: 0.99
Epoch: 7 Train Loss: tensor(0.0082) Train Accuracy: 0.9985714285714286 Test Accuracy: 0.98
Epoch: 8 Train Loss: tensor(0.0034) Train Accuracy: 0.9992857142857143 Test Accuracy: 0.99
Epoch: 9 Train Loss: tensor(0.0008) Train Accuracy: 0.9997619047619047 Test Accuracy: 0.98
